In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv('df_M30.csv', delimiter=';')

In [ ]:
new_df = df[df['oldID'].str.contains('PM1.*1$')]

In [ ]:
new_df = new_df.loc[new_df['oldID'].str.len() == 7]

In [ ]:
new_df['KM'] = new_df["oldID"].str[3:5].astype(str) + "." + new_df["oldID"].str[5:6].astype(str)

In [ ]:
new_df = new_df[new_df['fecha'].str.contains('2015-08-04')]

In [ ]:
new_df[['KM']] = new_df.loc[:,['KM']].astype(np.float64)

In [ ]:
new_df.loc[:, 'densidad'] = new_df.loc[:, "intensidad"] / new_df.loc[:, "vmed"]

In [ ]:
new_df.dropna(subset = ['densidad'])

In [ ]:
new_df['Hora'] = new_df["fecha"].str[11:13].astype(str) + "." + new_df["fecha"].str[14:16].astype(str)
new_df[['Hora']] = new_df.loc[:,['Hora']].astype(np.float64)

In [ ]:
new_df.sort_values('Hora').reset_index(drop=True)

In [ ]:
new_df.plot.scatter(x="densidad", y="Hora")

In [ ]:
display(new_df)

In [ ]:
new_df.dtypes

In [ ]:
new_df.to_csv('df_M30_KM_dia.csv', sep=';')

# Database of one day

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
# Leo el CSV y dibujo los scatter

df_un_dia = pd.read_csv('df_M30_KM_dia.csv', delimiter=';')
df_un_dia.plot.scatter(x="densidad", y="Hora")
df_un_dia.plot.scatter(x="Hora", y="KM")

In [ ]:
# Ordeno el dataframe por KM y Horas y quito las mediciones erróneas

df_prueba = df_un_dia.copy()
df_prueba = df_prueba.sort_values(by=['KM', 'Hora'])
df_prueba = df_prueba.reset_index(drop=True)
df_prueba = df_prueba[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df_prueba = df_prueba[~(df_prueba['KM'] == 0.2)]
df_prueba = df_prueba[~(df_prueba['KM'] == 0.9)]
df_prueba.index = range(len(df_prueba.index))

In [ ]:
# Genero un dataframe nuevo para las interpolaciones (15 minutos -> 1 minuto)

nans = np.where(np.empty_like(np.arange(576000).reshape(57600, 10)), np.nan, np.nan)
df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion', 
                                     'KM', 'densidad', 'Hora'])

In [ ]:
# Asigno los valores registrados cada 15 columnas y el resto como NaN para interpolar.

j = 0
for i in df_new.index.values.astype(int):
    print("i: ", i, "j: ", j)
    if (i%15) == 0:
        df_new.loc[i] = df_prueba.values[j]
        if i != 0:
            j=j+1
    else:
        df_new.loc[[i], ['ID']] = df_prueba.loc[[j], ['ID']]
        df_new.loc[[i], ['oldID']] = df_prueba.loc[[j], ['oldID']]
        df_new.loc[[i], ['fecha']] = df_prueba.loc[[j], ['fecha']]
        df_new.loc[[i], ['periodo_integracion']] = 1
        df_new.loc[[i], ['KM']] = df_prueba.loc[[j], ['KM']]

In [ ]:
# Exporto a CSV el DataFrame

df_new.to_csv('df_M30_KM_dia_perinterpol.csv', sep=';')

# Interpolado

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
df_new = pd.read_csv('df_M30_KM_dia_perinterpol.csv', delimiter=';')
#df_new.plot.scatter(x="Hora", y="densidad")
#df_new.plot.scatter(x="Hora", y="KM", c="vmed", colormap='viridis')

df_new = df_new.sort_values(by=['KM', 'Hora'])
df_new = df_new.reset_index(drop=True)
df_new = df_new[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df_new = df_new[~(df_new['KM'] == 0.2)]
df_new = df_new[~(df_new['KM'] == 0.9)]
df_new.index = range(len(df_new.index))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(df_new.KM, df_new.Hora, df_new.vmed, c=df_new.vmed, cmap='magma');

#Axes3D.scatter(xs=df_new.Hora, ys=df_new.KM, zs=df_new.vmed, zdir='z', **kwargs)

# Testing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import math
import decimal
%matplotlib inline

In [2]:
# Interpolo.

df = pd.read_csv('df_M30_KM_dia.csv', delimiter=',')

df = df.sort_values(by=['KM', 'Hora'])
df = df.reset_index(drop=True)
df = df[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df = df[~(df['KM'] == 0.2)]
df = df[~(df['KM'] == 0.9)]
df.index = range(len(df.index))

df_temp = pd.DataFrame([])
df_int = pd.DataFrame([])

for id in np.sort(df.oldID.value_counts().keys()):
    print("oldID: ", id)
    df_aux = df.loc[df['oldID'] == id]
    # Se crea la matriz de interpolación de tamaño: filas del id obtenido * 15 minutos interpolados
    #- 14 últimos minutos del día sin medición * 10 columnas
    nans = np.where(np.empty_like(np.arange((len(df_aux.index)*15-14)*10).reshape(len(df_aux.index)*15-14, 10)), np.nan, np.nan)
    df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion', 
                                         'KM', 'densidad', 'Hora'])
    j = 0
    for i in df_new.index.values.astype(int):
        if (i%15) == 0:
            if i != 0:
                j=j+1
            df_new.loc[i] = df_aux.values[j]
        else:
            df_new.iloc[i, df.columns.get_loc('ID')] = df_aux.iloc[j, df.columns.get_loc('ID')]
            df_new.iloc[i, df.columns.get_loc('oldID')] = df_aux.iloc[j, df.columns.get_loc('oldID')]
            #df_new.iloc[i, df.columns.get_loc('fecha')] = df_aux.iloc[j, df.columns.get_loc('fecha')]
            df_new.iloc[i, df.columns.get_loc('periodo_integracion')] = 1.0
            df_new.iloc[i, df.columns.get_loc('KM')] = df_aux.iloc[j, df.columns.get_loc('KM')]
    
    
    df_new['intensidad'] = df_new['intensidad'].interpolate()
    df_new['ocupacion'] = df_new['ocupacion'].interpolate()
    df_new['vmed'] = df_new['vmed'].interpolate()
    df_new['densidad'] = df_new['densidad'].interpolate()
    #df_int['Hora'] = df_new['Hora'].interpolate()
    if id == 'PM10001':
        df_temp = df_new
    else:
        df_temp = df_temp.append(df_new, ignore_index=True)

oldID:  PM10001
oldID:  PM10141
oldID:  PM10211
oldID:  PM10241
oldID:  PM10341
oldID:  PM10401
oldID:  PM10441
oldID:  PM10471
oldID:  PM10501
oldID:  PM10561
oldID:  PM10901
oldID:  PM10941
oldID:  PM10981
oldID:  PM11071
oldID:  PM11101
oldID:  PM11161
oldID:  PM11201
oldID:  PM11301
oldID:  PM11981
oldID:  PM12061
oldID:  PM12121
oldID:  PM12211
oldID:  PM12241
oldID:  PM12331
oldID:  PM12391
oldID:  PM12471
oldID:  PM12571
oldID:  PM12641
oldID:  PM12721
oldID:  PM12781
oldID:  PM12851
oldID:  PM12921
oldID:  PM12961
oldID:  PM12981
oldID:  PM13011
oldID:  PM13041
oldID:  PM13101
oldID:  PM13181
oldID:  PM13211
oldID:  PM13221


In [219]:
df_1 = pd.concat([df_temp['fecha']], axis=1, keys=['fecha'])

In [215]:
df_1['fecha'] = pd.to_datetime(df_1['fecha'])
df_1['hour'] = df_1.fecha.dt.hour
df_1['minute'] = df_1.fecha.dt.minute
df_1['fecha'] = df_1.fecha.dt.strftime('%d-%m-%Y')

#df_1['hour'] = df_1['hour'].interpolate()
for i in df_1.index.values.astype(int):
    if i != 0 and df_1.iloc[i]['minute'] == 0.0:
        #print("i:", (i))
        df_1.at[i-1,'minute'] =  59.0
    if i != 0 and (i%15) != 0:
        print("i:", i, "cp:", df_1.at[i-1,'hour'])
        df_1.at[i,'hour'] =  df_1.at[i-1,'hour']
    
df_1['minute'] = df_1['minute'].interpolate(method='linear')

i: 1 cp: 0.0
i: 2 cp: 0.0
i: 3 cp: 0.0
i: 4 cp: 0.0
i: 5 cp: 0.0
i: 6 cp: 0.0
i: 7 cp: 0.0
i: 8 cp: 0.0
i: 9 cp: 0.0
i: 10 cp: 0.0
i: 11 cp: 0.0
i: 12 cp: 0.0
i: 13 cp: 0.0
i: 14 cp: 0.0
i: 16 cp: 0.0
i: 17 cp: 0.0
i: 18 cp: 0.0
i: 19 cp: 0.0
i: 20 cp: 0.0
i: 21 cp: 0.0
i: 22 cp: 0.0
i: 23 cp: 0.0
i: 24 cp: 0.0
i: 25 cp: 0.0
i: 26 cp: 0.0
i: 27 cp: 0.0
i: 28 cp: 0.0
i: 29 cp: 0.0
i: 31 cp: 0.0
i: 32 cp: 0.0
i: 33 cp: 0.0
i: 34 cp: 0.0
i: 35 cp: 0.0
i: 36 cp: 0.0
i: 37 cp: 0.0
i: 38 cp: 0.0
i: 39 cp: 0.0
i: 40 cp: 0.0
i: 41 cp: 0.0
i: 42 cp: 0.0
i: 43 cp: 0.0
i: 44 cp: 0.0
i: 46 cp: 0.0
i: 47 cp: 0.0
i: 48 cp: 0.0
i: 49 cp: 0.0
i: 50 cp: 0.0
i: 51 cp: 0.0
i: 52 cp: 0.0
i: 53 cp: 0.0
i: 54 cp: 0.0
i: 55 cp: 0.0
i: 56 cp: 0.0
i: 57 cp: 0.0
i: 58 cp: 0.0
i: 59 cp: 0.0
i: 61 cp: 1.0
i: 62 cp: 1.0
i: 63 cp: 1.0
i: 64 cp: 1.0
i: 65 cp: 1.0
i: 66 cp: 1.0
i: 67 cp: 1.0
i: 68 cp: 1.0
i: 69 cp: 1.0
i: 70 cp: 1.0
i: 71 cp: 1.0
i: 72 cp: 1.0
i: 73 cp: 1.0
i: 74 cp: 1.0
i: 76 cp: 1.0
i: 77 cp: 1.0
i

i: 787 cp: 13.0
i: 788 cp: 13.0
i: 789 cp: 13.0
i: 790 cp: 13.0
i: 791 cp: 13.0
i: 792 cp: 13.0
i: 793 cp: 13.0
i: 794 cp: 13.0
i: 796 cp: 13.0
i: 797 cp: 13.0
i: 798 cp: 13.0
i: 799 cp: 13.0
i: 800 cp: 13.0
i: 801 cp: 13.0
i: 802 cp: 13.0
i: 803 cp: 13.0
i: 804 cp: 13.0
i: 805 cp: 13.0
i: 806 cp: 13.0
i: 807 cp: 13.0
i: 808 cp: 13.0
i: 809 cp: 13.0
i: 811 cp: 13.0
i: 812 cp: 13.0
i: 813 cp: 13.0
i: 814 cp: 13.0
i: 815 cp: 13.0
i: 816 cp: 13.0
i: 817 cp: 13.0
i: 818 cp: 13.0
i: 819 cp: 13.0
i: 820 cp: 13.0
i: 821 cp: 13.0
i: 822 cp: 13.0
i: 823 cp: 13.0
i: 824 cp: 13.0
i: 826 cp: 13.0
i: 827 cp: 13.0
i: 828 cp: 13.0
i: 829 cp: 13.0
i: 830 cp: 13.0
i: 831 cp: 13.0
i: 832 cp: 13.0
i: 833 cp: 13.0
i: 834 cp: 13.0
i: 835 cp: 13.0
i: 836 cp: 13.0
i: 837 cp: 13.0
i: 838 cp: 13.0
i: 839 cp: 13.0
i: 841 cp: 14.0
i: 842 cp: 14.0
i: 843 cp: 14.0
i: 844 cp: 14.0
i: 845 cp: 14.0
i: 846 cp: 14.0
i: 847 cp: 14.0
i: 848 cp: 14.0
i: 849 cp: 14.0
i: 850 cp: 14.0
i: 851 cp: 14.0
i: 852 cp: 14.0
i: 853 c

i: 1467 cp: nan
i: 1468 cp: nan
i: 1469 cp: nan
i: 1471 cp: nan
i: 1472 cp: nan
i: 1473 cp: nan
i: 1474 cp: nan
i: 1475 cp: nan
i: 1476 cp: nan
i: 1477 cp: nan
i: 1478 cp: nan
i: 1479 cp: nan
i: 1480 cp: nan
i: 1481 cp: nan
i: 1482 cp: nan
i: 1483 cp: nan
i: 1484 cp: nan
i: 1486 cp: nan
i: 1487 cp: nan
i: 1488 cp: nan
i: 1489 cp: nan
i: 1490 cp: nan
i: 1491 cp: nan
i: 1492 cp: nan
i: 1493 cp: nan
i: 1494 cp: nan
i: 1495 cp: nan
i: 1496 cp: nan
i: 1497 cp: nan
i: 1498 cp: nan
i: 1499 cp: nan
i: 1501 cp: nan
i: 1502 cp: nan
i: 1503 cp: nan
i: 1504 cp: nan
i: 1505 cp: nan
i: 1506 cp: nan
i: 1507 cp: nan
i: 1508 cp: nan
i: 1509 cp: nan
i: 1510 cp: nan
i: 1511 cp: nan
i: 1512 cp: nan
i: 1513 cp: nan
i: 1514 cp: nan
i: 1516 cp: nan
i: 1517 cp: nan
i: 1518 cp: nan
i: 1519 cp: nan
i: 1520 cp: nan
i: 1521 cp: nan
i: 1522 cp: nan
i: 1523 cp: nan
i: 1524 cp: nan
i: 1525 cp: nan
i: 1526 cp: nan
i: 1527 cp: nan
i: 1528 cp: nan
i: 1529 cp: nan
i: 1531 cp: nan
i: 1532 cp: nan
i: 1533 cp: nan
i: 1534 

i: 2050 cp: nan
i: 2051 cp: nan
i: 2052 cp: nan
i: 2053 cp: nan
i: 2054 cp: nan
i: 2056 cp: nan
i: 2057 cp: nan
i: 2058 cp: nan
i: 2059 cp: nan
i: 2060 cp: nan
i: 2061 cp: nan
i: 2062 cp: nan
i: 2063 cp: nan
i: 2064 cp: nan
i: 2065 cp: nan
i: 2066 cp: nan
i: 2067 cp: nan
i: 2068 cp: nan
i: 2069 cp: nan
i: 2071 cp: nan
i: 2072 cp: nan
i: 2073 cp: nan
i: 2074 cp: nan
i: 2075 cp: nan
i: 2076 cp: nan
i: 2077 cp: nan
i: 2078 cp: nan
i: 2079 cp: nan
i: 2080 cp: nan
i: 2081 cp: nan
i: 2082 cp: nan
i: 2083 cp: nan
i: 2084 cp: nan
i: 2086 cp: nan
i: 2087 cp: nan
i: 2088 cp: nan
i: 2089 cp: nan
i: 2090 cp: nan
i: 2091 cp: nan
i: 2092 cp: nan
i: 2093 cp: nan
i: 2094 cp: nan
i: 2095 cp: nan
i: 2096 cp: nan
i: 2097 cp: nan
i: 2098 cp: nan
i: 2099 cp: nan
i: 2101 cp: nan
i: 2102 cp: nan
i: 2103 cp: nan
i: 2104 cp: nan
i: 2105 cp: nan
i: 2106 cp: nan
i: 2107 cp: nan
i: 2108 cp: nan
i: 2109 cp: nan
i: 2110 cp: nan
i: 2111 cp: nan
i: 2112 cp: nan
i: 2113 cp: nan
i: 2114 cp: nan
i: 2116 cp: nan
i: 2117 

i: 2801 cp: nan
i: 2802 cp: nan
i: 2803 cp: nan
i: 2804 cp: nan
i: 2806 cp: nan
i: 2807 cp: nan
i: 2808 cp: nan
i: 2809 cp: nan
i: 2810 cp: nan
i: 2811 cp: nan
i: 2812 cp: nan
i: 2813 cp: nan
i: 2814 cp: nan
i: 2815 cp: nan
i: 2816 cp: nan
i: 2817 cp: nan
i: 2818 cp: nan
i: 2819 cp: nan
i: 2821 cp: nan
i: 2822 cp: nan
i: 2823 cp: nan
i: 2824 cp: nan
i: 2825 cp: nan
i: 2826 cp: nan
i: 2827 cp: nan
i: 2828 cp: nan
i: 2829 cp: nan
i: 2830 cp: nan
i: 2831 cp: nan
i: 2832 cp: nan
i: 2833 cp: nan
i: 2834 cp: nan
i: 2836 cp: nan
i: 2837 cp: nan
i: 2838 cp: nan
i: 2839 cp: nan
i: 2840 cp: nan
i: 2841 cp: nan
i: 2842 cp: nan
i: 2843 cp: nan
i: 2844 cp: nan
i: 2845 cp: nan
i: 2846 cp: nan
i: 2847 cp: nan
i: 2848 cp: nan
i: 2849 cp: nan
i: 2851 cp: nan
i: 2852 cp: nan
i: 2853 cp: nan
i: 2854 cp: nan
i: 2855 cp: nan
i: 2856 cp: nan
i: 2857 cp: nan
i: 2858 cp: nan
i: 2859 cp: nan
i: 2860 cp: nan
i: 2861 cp: nan
i: 2862 cp: nan
i: 2863 cp: nan
i: 2864 cp: nan
i: 2866 cp: nan
i: 2867 cp: nan
i: 2868 

...

i: 54385 cp: nan
i: 54386 cp: nan
i: 54387 cp: nan
i: 54388 cp: nan
i: 54389 cp: nan
i: 54391 cp: nan
i: 54392 cp: nan
i: 54393 cp: nan
i: 54394 cp: nan
i: 54395 cp: nan
i: 54396 cp: nan
i: 54397 cp: nan
i: 54398 cp: nan
i: 54399 cp: nan
i: 54400 cp: nan
i: 54401 cp: nan
i: 54402 cp: nan
i: 54403 cp: nan
i: 54404 cp: nan
i: 54406 cp: nan
i: 54407 cp: nan
i: 54408 cp: nan
i: 54409 cp: nan
i: 54410 cp: nan
i: 54411 cp: nan
i: 54412 cp: nan
i: 54413 cp: nan
i: 54414 cp: nan
i: 54415 cp: nan
i: 54416 cp: nan
i: 54417 cp: nan
i: 54418 cp: nan
i: 54419 cp: nan
i: 54421 cp: nan
i: 54422 cp: nan
i: 54423 cp: nan
i: 54424 cp: nan
i: 54425 cp: nan
i: 54426 cp: nan
i: 54427 cp: nan
i: 54428 cp: nan
i: 54429 cp: nan
i: 54430 cp: nan
i: 54431 cp: nan
i: 54432 cp: nan
i: 54433 cp: nan
i: 54434 cp: nan
i: 54436 cp: nan
i: 54437 cp: nan
i: 54438 cp: nan
i: 54439 cp: nan
i: 54440 cp: nan
i: 54441 cp: nan
i: 54442 cp: nan
i: 54443 cp: nan
i: 54444 cp: nan
i: 54445 cp: nan
i: 54446 cp: nan
i: 54447 cp: n

i: 55061 cp: nan
i: 55062 cp: nan
i: 55063 cp: nan
i: 55064 cp: nan
i: 55066 cp: nan
i: 55067 cp: nan
i: 55068 cp: nan
i: 55069 cp: nan
i: 55070 cp: nan
i: 55071 cp: nan
i: 55072 cp: nan
i: 55073 cp: nan
i: 55074 cp: nan
i: 55075 cp: nan
i: 55076 cp: nan
i: 55077 cp: nan
i: 55078 cp: nan
i: 55079 cp: nan
i: 55081 cp: nan
i: 55082 cp: nan
i: 55083 cp: nan
i: 55084 cp: nan
i: 55085 cp: nan
i: 55086 cp: nan
i: 55087 cp: nan
i: 55088 cp: nan
i: 55089 cp: nan
i: 55090 cp: nan
i: 55091 cp: nan
i: 55092 cp: nan
i: 55093 cp: nan
i: 55094 cp: nan
i: 55096 cp: nan
i: 55097 cp: nan
i: 55098 cp: nan
i: 55099 cp: nan
i: 55100 cp: nan
i: 55101 cp: nan
i: 55102 cp: nan
i: 55103 cp: nan
i: 55104 cp: nan
i: 55105 cp: nan
i: 55106 cp: nan
i: 55107 cp: nan
i: 55108 cp: nan
i: 55109 cp: nan
i: 55111 cp: nan
i: 55112 cp: nan
i: 55113 cp: nan
i: 55114 cp: nan
i: 55115 cp: nan
i: 55116 cp: nan
i: 55117 cp: nan
i: 55118 cp: nan
i: 55119 cp: nan
i: 55120 cp: nan
i: 55121 cp: nan
i: 55122 cp: nan
i: 55123 cp: n

i: 55696 cp: nan
i: 55697 cp: nan
i: 55698 cp: nan
i: 55699 cp: nan
i: 55700 cp: nan
i: 55701 cp: nan
i: 55702 cp: nan
i: 55703 cp: nan
i: 55704 cp: nan
i: 55705 cp: nan
i: 55706 cp: nan
i: 55707 cp: nan
i: 55708 cp: nan
i: 55709 cp: nan
i: 55711 cp: nan
i: 55712 cp: nan
i: 55713 cp: nan
i: 55714 cp: nan
i: 55715 cp: nan
i: 55716 cp: nan
i: 55717 cp: nan
i: 55718 cp: nan
i: 55719 cp: nan
i: 55720 cp: nan
i: 55721 cp: nan
i: 55722 cp: nan
i: 55723 cp: nan
i: 55724 cp: nan
i: 55726 cp: nan
i: 55727 cp: nan
i: 55728 cp: nan
i: 55729 cp: nan
i: 55730 cp: nan
i: 55731 cp: nan
i: 55732 cp: nan
i: 55733 cp: nan
i: 55734 cp: nan
i: 55735 cp: nan
i: 55736 cp: nan
i: 55737 cp: nan
i: 55738 cp: nan
i: 55739 cp: nan
i: 55741 cp: nan
i: 55742 cp: nan
i: 55743 cp: nan
i: 55744 cp: nan
i: 55745 cp: nan
i: 55746 cp: nan
i: 55747 cp: nan
i: 55748 cp: nan
i: 55749 cp: nan
i: 55750 cp: nan
i: 55751 cp: nan
i: 55752 cp: nan
i: 55753 cp: nan
i: 55754 cp: nan
i: 55756 cp: nan
i: 55757 cp: nan
i: 55758 cp: n

i: 56260 cp: nan
i: 56261 cp: nan
i: 56262 cp: nan
i: 56263 cp: nan
i: 56264 cp: nan
i: 56266 cp: nan
i: 56267 cp: nan
i: 56268 cp: nan
i: 56269 cp: nan
i: 56270 cp: nan
i: 56271 cp: nan
i: 56272 cp: nan
i: 56273 cp: nan
i: 56274 cp: nan
i: 56275 cp: nan
i: 56276 cp: nan
i: 56277 cp: nan
i: 56278 cp: nan
i: 56279 cp: nan
i: 56281 cp: nan
i: 56282 cp: nan
i: 56283 cp: nan
i: 56284 cp: nan
i: 56285 cp: nan
i: 56286 cp: nan
i: 56287 cp: nan
i: 56288 cp: nan
i: 56289 cp: nan
i: 56290 cp: nan
i: 56291 cp: nan
i: 56292 cp: nan
i: 56293 cp: nan
i: 56294 cp: nan
i: 56296 cp: nan
i: 56297 cp: nan
i: 56298 cp: nan
i: 56299 cp: nan
i: 56300 cp: nan
i: 56301 cp: nan
i: 56302 cp: nan
i: 56303 cp: nan
i: 56304 cp: nan
i: 56305 cp: nan
i: 56306 cp: nan
i: 56307 cp: nan
i: 56308 cp: nan
i: 56309 cp: nan
i: 56311 cp: nan
i: 56312 cp: nan
i: 56313 cp: nan
i: 56314 cp: nan
i: 56315 cp: nan
i: 56316 cp: nan
i: 56317 cp: nan
i: 56318 cp: nan
i: 56319 cp: nan
i: 56320 cp: nan
i: 56321 cp: nan
i: 56322 cp: n

i: 56900 cp: nan
i: 56901 cp: nan
i: 56902 cp: nan
i: 56903 cp: nan
i: 56904 cp: nan
i: 56905 cp: nan
i: 56906 cp: nan
i: 56907 cp: nan
i: 56908 cp: nan
i: 56909 cp: nan
i: 56911 cp: nan
i: 56912 cp: nan
i: 56913 cp: nan
i: 56914 cp: nan
i: 56915 cp: nan
i: 56916 cp: nan
i: 56917 cp: nan
i: 56918 cp: nan
i: 56919 cp: nan
i: 56920 cp: nan
i: 56921 cp: nan
i: 56922 cp: nan
i: 56923 cp: nan
i: 56924 cp: nan
i: 56926 cp: nan
i: 56927 cp: nan
i: 56928 cp: nan
i: 56929 cp: nan
i: 56930 cp: nan
i: 56931 cp: nan
i: 56932 cp: nan
i: 56933 cp: nan
i: 56934 cp: nan
i: 56935 cp: nan
i: 56936 cp: nan
i: 56937 cp: nan
i: 56938 cp: nan
i: 56939 cp: nan
i: 56941 cp: nan
i: 56942 cp: nan
i: 56943 cp: nan
i: 56944 cp: nan
i: 56945 cp: nan
i: 56946 cp: nan
i: 56947 cp: nan
i: 56948 cp: nan
i: 56949 cp: nan
i: 56950 cp: nan
i: 56951 cp: nan
i: 56952 cp: nan
i: 56953 cp: nan
i: 56954 cp: nan
i: 56956 cp: nan
i: 56957 cp: nan
i: 56958 cp: nan
i: 56959 cp: nan
i: 56960 cp: nan
i: 56961 cp: nan
i: 56962 cp: n

In [222]:
#df_1['hour'].unique()
df_1.isnull()
df_1[df_1.isnull().any(axis=1)]
df_1.loc[2840:2860]

,fecha
2840,NaN
2841,NaN
2842,NaN
2843,NaN
2844,NaN
2845,NaN
2846,NaN
2847,NaN
2848,NaN
2849,NaN


In [ ]:
# Exporto a CSV el DataFrame

df_good.to_csv('df_M30_KM_dia_inter_temp.csv', sep=';')

In [ ]:
df_good = pd.read_csv('df_M30_KM_dia_inter_temp.csv', delimiter=';')

In [ ]:
df_esp = pd.DataFrame([])
df_int = pd.DataFrame([])

df_good.Hora.round(2)

for time in np.sort(df_good.Hora.value_counts().keys()):
    print("Hora:", time)
    df_aux = df_good.loc[df_good['Hora'] == time]
    df_aux = df_aux.sort_values(by=['KM'])
    df_aux = df_aux.reset_index(drop=True)
    df_aux = df_aux.drop(columns=['Unnamed: 0'])
    
    nans = np.where(np.empty_like(np.arange(325*10).reshape(325, 10)), np.nan, np.nan)
    df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion','KM','densidad','Hora'])
    
    df_new['ID'] = 'int_temp'
    df_new['oldID'] = 'int_temp'
    df_new['Hora'] = time
    df_new['fecha'] = df_aux.iloc[0, df_aux.columns.get_loc('fecha')]
    df_new['periodo_integracion'] = 1.0
    
    #for km in df_aux['KM'].diff():
        #print("KM:", km)
    #    if not math.isnan(km):
            #print("km:", round(km, 1), "Size:", int(round((km/0.1)*10,1)), "Shape:", int(round(km/0.1,1)))
    #        nans = np.where(np.empty_like(np.arange(int(round((km/0.1)*10, 1))).reshape(int(round((km/0.1), 1)), 10)), np.nan, np.nan)
    #        df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion','KM','densidad','Hora'])
            
    #        df_new['ID'] = 'int_temp'
    #        df_new['oldID'] = 'int_temp'
    #        df_new['Hora'] = time
    #        df_new['fecha'] = df_aux.iloc[0, df_aux.columns.get_loc('fecha')]
    #        df_new['periodo_integracion'] = 1.0
            
    #        df_int = df_int.append(df_new, ignore_index=True)
    
    #print("Assigning values")
    j=0
    k = 0
    for i in df_new.index.values.astype(int):
        print("i:", i, "KM:", int(round(np.sort(df_good.KM.value_counts().keys())[k]/0.1, 1)))
        if i == int(round(np.sort(df_good.KM.value_counts().keys())[k]/0.1, 1)):
            print("Asigned")
            df_new.loc[i] = df_aux.loc[j].values
            j += 1
            k += 1
            
    print("Interpolating")
    #df_int_esp = df_int.interpolate()
    #if id == 'PM10001':
        #df_esp = df_int_esp
    #else:
        #df_esp = df_esp.append(df_int_esp, ignore_index=True)

In [ ]:
df_good

In [ ]:
df_esp.plot.scatter(x="Hora", y="densidad")
df_esp.plot.scatter(x="Hora", y="KM", c="vmed", colormap='viridis')